In [1]:
from mylib import preprocess
import pandas as pd
import os

pretest = pd.read_csv(os.path.join('data', 'test_users.csv'), header=0, parse_dates=[1,2,3])
pretrain = pd.read_csv(os.path.join('data', 'train_users_2.csv'), header=0, parse_dates=[1,2,3])
df_sessions = pd.read_csv("data\sessions.csv", encoding='utf8')

In [2]:
train, test, y, le = preprocess.make_user_features(pretrain, pretest)

In [3]:
data = pd.concat((train, test), axis=0, ignore_index=True)
final = preprocess.make_sessions_features(data, df_sessions)

In [4]:
final[:train.shape[0]].shape

(213451, 549)

In [5]:
from sklearn.cross_validation import train_test_split
our_train = final[:train.shape[0]]
X_train, X_test, y_train, y_test = train_test_split(our_train, y, test_size=0.33, random_state=42, stratify=y)

In [6]:
X_train.shape

(143012, 549)

In [7]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
clf = RandomForestClassifier(n_estimators=160, oob_score=True, n_jobs=-1, criterion='entropy')
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=160, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [8]:
import numpy as np
np.count_nonzero(clf.feature_importances_ < 1e-4)

312

In [9]:
unimportant_features = clf.feature_importances_ < 1e-4
clean_X_train = X_train.ix[:, ~unimportant_features]

In [10]:
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from pprint import pprint
bagg = AdaBoostClassifier(random_state=42)

param_grid = {"n_estimators": [10, 50, 100],
              "learning_rate": [0.1, 1.0, 10, 100]}

search = GridSearchCV(bagg, param_grid, scoring='f1_weighted', cv=3, verbose=4, n_jobs=2)
search.fit(clean_X_train, y_train)
pprint(sorted(search.grid_scores_, key=lambda x: -x.mean_validation_score))

[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:  3.6min
[Parallel(n_jobs=2)]: Done  36 out of  36 | elapsed:  5.9min finished


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[mean: 0.55355, std: 0.00245, params: {'n_estimators': 100, 'learning_rate': 1.0},
 mean: 0.54898, std: 0.00310, params: {'n_estimators': 50, 'learning_rate': 1.0},
 mean: 0.52288, std: 0.00815, params: {'n_estimators': 10, 'learning_rate': 1.0},
 mean: 0.47490, std: 0.00127, params: {'n_estimators': 100, 'learning_rate': 0.1},
 mean: 0.46160, std: 0.00067, params: {'n_estimators': 50, 'learning_rate': 0.1},
 mean: 0.45722, std: 0.00324, params: {'n_estimators': 10, 'learning_rate': 0.1},
 mean: 0.14334, std: 0.20270, params: {'n_estimators': 50, 'learning_rate': 100},
 mean: 0.14334, std: 0.20270, params: {'n_estimators': 100, 'learning_rate': 100},
 mean: 0.09136, std: 0.05664, params: {'n_estimators': 10, 'learning_rate': 10},
 mean: 0.08797, std: 0.05885, params: {'n_estimators': 100, 'learning_rate': 10},
 mean: 0.04407, std: 0.06230, params: {'n_estimators': 10, 'learning_rate': 100},
 mean: 0.01193, std: 0.01395, param